In [3]:
# 모델 매개변수 최적화하기

# Process
# 1. 모델과 데이터 준비
# 2. 데이터에 매개변수를 최적화
# 3. 모델을 학습, 검증 ,테스트 

# 모델을 학습하는 것은 반복적인 과정을 거친다.
# 각 반복 단계에서 모델은 출력을 추측,
# 그리고 추측과 정답 사이의 loss를 계산
# 매개변수에 대한 오류의 도함수를 수집
# 경사하강법을 사용하여 이 파리미터들을 optimize

In [4]:
# Basic Code from Dataloader
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
# hyperparameter : 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
# 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다
# 학습 시에는 다음과 같은 하이퍼파라미터를 정의:
# epoch - 데이터셋을 반복하는 횟수
# batch size - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
# learning rate - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 
# 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
# 최적화 단계(Optimization Loop)
# 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 
# 최적화 단계의 각 반복(iteration)을 에폭이라고 부른다.

# 하나의 에폭은 두 부분으로 구성된다.
# 1. 학습 단계 : 학습용 데이터셋을 반복하고, 최적의 매개변수로 수렴
# 2. 검증/테스트 단계 : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복


# 학습 단계에서 일어나는 Process

# loss function 
# 획득한 결과와 실제 값 사이의 틀린 정도를 계산 -> 학습 중에 이 값을 최소화 하려고 한다.
# 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산
# nn.MSELoss, nn.NLLLoss(negative log likehood), nn.CrossEntropyLoss(nn.LogSoftmax+nn.NLLloss) 등이 있음

# 모델의 출력 로짓(logit)을 nn.CrossEntropyLoss에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산
# 손실 함수를 초기화
loss_fn = nn.CrossEntropyLoss()

In [7]:
# Optimizer
# 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
# 모든 최적화 절차(logic)는 optimizer 객체에 캡슐화(encapsulate)
# SGD 옵티마이저를 사용
# PyTorch에는 ADAM이나 RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있다.

# 학습하려는 모델의 매개변수와 학습률 하이퍼파리미터를 등록하여 옵티마이저를 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 학습 단계에서 최적화는 세단계로 이루어진다.
# 1. optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정
# (기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정)
# 2. loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파. 
# PyTorch는 각 매개변수에 대한 손실의 변화도를 저장
# 3. 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정



In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 모델을 학습(train) 모드로 설정합니다 - 배치 정규화(Batch Normalization) 및 드롭아웃(Dropout) 레이어들에 중요합니다.
    # 이 예시에서는 없어도 되지만, 모범 사례를 위해 추가해두었습니다.
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 모델을 평가(eval) 모드로 설정합니다 - 배치 정규화(Batch Normalization) 및 드롭아웃(Dropout) 레이어들에 중요합니다.
    # 이 예시에서는 없어도 되지만, 모범 사례를 위해 추가해두었습니다.
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # torch.no_grad()를 사용하여 테스트 시 변화도(gradient)를 계산하지 않도록 합니다.
    # 이는 requires_grad=True로 설정된 텐서들의 불필요한 변화도 연산 및 메모리 사용량 또한 줄여줍니다.
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291257  [   64/60000]
loss: 2.277385  [ 6464/60000]
loss: 2.260304  [12864/60000]
loss: 2.269151  [19264/60000]
loss: 2.227650  [25664/60000]
loss: 2.208390  [32064/60000]
loss: 2.218048  [38464/60000]
loss: 2.179694  [44864/60000]
loss: 2.170037  [51264/60000]
loss: 2.135387  [57664/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.127222 

Epoch 2
-------------------------------
loss: 2.136223  [   64/60000]
loss: 2.118251  [ 6464/60000]
loss: 2.058158  [12864/60000]
loss: 2.081075  [19264/60000]
loss: 2.003078  [25664/60000]
loss: 1.958703  [32064/60000]
loss: 1.977543  [38464/60000]
loss: 1.898535  [44864/60000]
loss: 1.896061  [51264/60000]
loss: 1.810694  [57664/60000]
Test Error: 
 Accuracy: 58.4%, Avg loss: 1.816308 

Epoch 3
-------------------------------
loss: 1.856323  [   64/60000]
loss: 1.812883  [ 6464/60000]
loss: 1.699240  [12864/60000]
loss: 1.740832  [19264/60000]
loss: 1.618805  [25664/60000]
loss: 1.598148  [32064/600